# Transformation of intensities

Transformations of intensities — behold position of the pixels, but change their intensities.

We will consider two types of transformations:
*  Pixel wise
*  Area wise (filtration)

Let's start with pixel wise transformations.

## Pixelwise transformations

In this case we change the intensity of every pixel by applying different mathematical functions or operators. They key point is that the same operation is applied to every pixel. 

### Conversion between color models

The simples transformation with intensities is a conversion between different color models. For example, you can take a color image represented by RGB (Red, Green, Blue) model and make it  grayscale. It's commonly used for simplifying images or preparing them for processing tasks.

Here is an example:

In [ ]:
# load Apple image from file
import numpy as np
from PIL import Image

img = Image.open("datasets/Apples.jpg")
img

In [ ]:
# show size of image and size of corresponding array
img_arr = np.array(img)
(img.size, img_arr.shape)

Show elements of the array:

In [ ]:
img_arr

In [ ]:
# Converting an image to grayscale
gray_img = img.convert("L")
gray_img

In [ ]:
# show size of image and size of corresponding array
gray_img_arr = np.array(gray_img)
(gray_img.size, gray_img_arr.shape)

In [ ]:
# show elements of array
gray_img_arr

As you can see the gray scale image is represented by 2D array, not 3D, as we expected.

There is also a possibility to convert to a binary image, with only two values. We will discuss it a bit later in the class.

### Point operations

Point operations let you apply any function to compute a new intensity of each pixel. If image is color it will apply this function to each channel. For example, in the code below we do a threshold operation — compare intensity of every pixel on grayscale image with 128 and return either 0 (if intensity is below or equal to 128) or 255 (if it is above) as a result.

In [ ]:
# compare intensity of every pixel with 128 and return either 255 or 0
bw_img = gray_img.point(lambda p: 255 if p > 128 else 0)
bw_img

You can do a more sophisticated operations, e.g. use exponential or logarithmic functions to change intensity of dark and light parts of the image.

You can also apply this function to color image, every color channel will be processed the same way:

In [ ]:
# apply threshold to the color image
img_thresholded = img.point(lambda p: 255 if p > 128 else 0)
img_thresholded

In general, it is easier to do point wise operations directly with arrays, as this will give more flexibility. Just remember, that some of the operations will result in floating point numbers outside the [0, 255] range, so you have to rescale the intensities and convert to integer.

Just a quick example:

In [ ]:
# convert image to NumPy array
img_arr_in = np.array(img)

# create empty array for the processed image
img_arr_out = np.zeros(img_arr_in.shape, dtype=int)

# loop over color channels
for c in range(0, 3):

    # get slice of array corresponding to a given channel
    x = img_arr_in[:, :, c]

    # compute logarithm of intensities (we add 0.1 to avoid log(0))
    x = np.log(x + 0.1)

    # rescale the intensities so they will be between 0 and 1
    x = (x - x.min()) / (x.max() - x.min())

    # multiply the rescaled intensities to 255 and save to array
    img_arr_out[:, :, c] = x * 255

# convert the array to Image object and show
img_out = Image.fromarray(np.uint8(img_arr_out))
img_out

Try to play with the code but using different functions instead of logarithm (e.g. power, square root, etc.).

## Filtering/convolution

Another way to change the intensity of pixels is to make them dependent on their neighbors. This transformation in general is called *filtering*. If filtering is implemented by taking a weighted sum of intensities in a given neighborhood this operation is called a *convolution* and the weights form a convolutional *kernel* or just a *filter*.

**Open the third sheet of the [Excel file](./mlcourse.xlsm) and play with filtering example, then come back.**

There are many filters available, some of them have practical meaning, e.g. removing noise on images, some of them are used mostly for creating funny effects. 

One of the most common family of filters is *blurring* filters — they blur fine details on an image. One of the simplest is median filter, which computes new intensity of every pixel by taking median of intensities of its neighbors. 

Here is an example with filter size 7 x 7 so the median intensity will be computed for a given pixel and its 48 neighbors around it:

In [ ]:
# load class which contains all filters from PIL
from PIL import ImageFilter

# apply median filter with window 7x7
blurred_img = img.filter(ImageFilter.MedianFilter(size=7))
blurred_img

PIL/Pillow contains a lot of predefined filters, you can see them by typing `ImageFilter.` so Jupyter notebook will show everything this class contains, including methods and predefined values. The predefined filters are named using capital letters, like:

```python
ImageFilter.BLUR
ImageFilter.EDGE_ENHANCE
```

There are also several functions which can construct filters using various additional parameters, like you already tried above (median filter). Here is another one:

```python
f = ImageFilter.GaussianBlur(radius=2)
```

Here is an example of using Gaussian blur:

In [ ]:
img_gb = img.filter(ImageFilter.GaussianBlur(radius=2))
img_gb

As you can see in this case it blurs both edges and surfaces of apples while the median filter keeps the edges sharp.

You can also create your own filter by providing weights of the convolutional kernel. For example the following filter will enhance vertical lines on the image:

In [ ]:
# create manual filter (size, weights by row, scale, offset)
f = ImageFilter.Kernel((3, 3), (-1, 0, 1, -1, 0, 1, -1, 0, 1), 1, 0)

# apply and show the result
img_filtered = img.filter(f)
img_filtered

## Batch processing

Of course you can apply any of the transformations together and to a large number of images simultaneously. This is usually called as *batch processing*. In order to do that follow the guidelines:

1. Create a function which applies all transformation you need to one single image.
2. Put all images that you want to transform to a dedicated folder.
3. Create an empty folder for the transformed images.

After that let Python do the job for you. It has a specific library, `os`, which can work with files, read a list of files from a directory, etc. 

Let's first write a transformation function which creates a mask (new image consisting of 0 and 255 only). And then multiplies the original image to the mask, crop the result, and return it. Check how it works:


In [ ]:
from PIL import ImageFilter, ImageChops

def transform(img):

    # convert image to grayscale and save to new image
    mask = img.convert("L")

    # apply median blur
    mask = mask.filter(ImageFilter.MedianFilter(size=9))

    # apply threshold and convert back to RGB
    mask = mask.point(lambda p: 255 if p < 150 else 0)
    mask = mask.convert("RGB")

    # multiply the original image to mask
    new_img =  ImageChops.multiply(img, mask)

    # crop 20% around the new image
    width, height = img.size

    left = int(width * 0.20)
    right = left + int(width * 0.60)
    top = int(height * 0.20)
    bottom = top + int(height * 0.60)

    new_img = new_img.crop((left, top, right, bottom))

    return new_img

Let's apply the function to another image, `IG_101958.jpg`, which is located in folder `datasets/images`. Here is the original image:

In [ ]:
img = Image.open("datasets/images/IG_101958.jpg")
img

And here is the result of transformation:

In [ ]:
img_transformed = transform(img)
img_transformed


Now let's learn how to use library `os` to get all files from a specific folder:

In [ ]:
import os

# defining a directory containing images
img_dir = "datasets/images/"

# listing all files in the directory
img_files = os.listdir(img_dir)

img_files

As you can see, the method `os.listdir()` returns a list of all files located inside this folder. Now we can loop over the files and do the batch processing:

In [ ]:
from PIL import Image
import os

# Specifying your input and output folders
input_folder = "datasets/images/"
output_folder = "datasets/processed-images/"

# making sure the output folder exists, or create it
os.makedirs(output_folder, exist_ok=True)


In [ ]:
files = os.listdir(input_folder)

# looping through each file in the input folder
for filename in files:

    # check if file is JPEG image and if so do processing
    if filename.endswith(".jpg"):

        # make full path to the input and output image
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        # read the image from file
        img = Image.open(input_path)

        # apply transformations
        new_img = transform(img)

        # save the transformed image to the output folder
        new_img.save(output_path)

Run it and you will see that you got a new folder inside `datasets` — `processed images`. It contains the same files as in `images` but if you compare them you will see that all files in the new folder contain the transformed images.

### Exercise

Play with the transformation code to improve segmentation results.  